# Uploading objects to AIS with the SDK

In [ ]:
pip install aistore

In [ ]:
from aistore import Client

# Use the client class to get a reference to a named bucket:
ais_url = "http://localhost:8080"
client = Client(ais_url)

# All object operations are done within the context of a bucket
bucket = client.bucket("my-bck")

## There are 4 primary ways to create objects in AIS with the SDK
1. object.put_content creates an object with raw bytes
2. object.put_file creates an object from file contents
3. object.promote takes files the AIS storage targets can access and promotes them to objects
4. bucket.put_files puts an entire directory as multiple objects in AIS

### Object.put_data

In [ ]:
data_obj = bucket.object("my-data-object")
data_obj.put_content(b"raw bytes content")
bucket.list_objects()

### Object.put_file

In [ ]:
import tempfile

# Create a temporary file to demonstrate put_file
with tempfile.NamedTemporaryFile() as f:
    f.write(b"content inside of a local file")
    f.flush()
    bucket.object("my-file-object").put_file(f.name)
bucket.list_objects()

### Object.promote

This method only works if the filepath provided can be accessed by the AIS storage targets. See [the aiatscale.org blog](https://aiatscale.org/blog/2022/03/17/promote) for details on promoting.
It will work in a local example if AIS is running on the same machine, shown below.

In [ ]:
from pathlib import Path
import shutil

# Set up an example folder with multiple files
example_dir = Path().absolute().joinpath("promote-example")
example_dir.mkdir(exist_ok=True)
filenames = [f"file_{i}" for i in range(10)]
for filename in filenames:
    filepath = example_dir.joinpath(filename)
    with open(filepath, "w") as file:
        file.write("content in each test file")
# Promote the entire directory (more options available in the docs)
bucket.object("promoted-object").promote(str(example_dir))

# Delete the example folder
shutil.rmtree(example_dir)

bucket.list_objects()

### Bucket.put_files

In [ ]:
# Again, set up an example folder with multiple files
example_dir = Path().absolute().joinpath("put_files-example")
example_dir.mkdir(exist_ok=True)
filenames = [f"file_{i}" for i in range(10)]
for filename in filenames:
    filepath = example_dir.joinpath(filename)
    with open(filepath, "w") as file:
        file.write("content in each test file")
# Put each file in the entire directory (more options available in the docs, including recursive put)
bucket.put_files(str(example_dir), obj_prefix="example-put-")

# Delete the example folder
shutil.rmtree(example_dir)

bucket.list_objects()